<a href="https://colab.research.google.com/github/dmnarula/football-ds-learning/blob/main/week3-visualization/Week3_Projects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd

url = "https://raw.githubusercontent.com/dmnarula/football-ds-learning/refs/heads/main/week3-visualization/data/pl_data_gw14_25_26_full.csv"
df = pd.read_csv(url)

df.head()

,id,status,chance_of_playing_next_round,chance_of_playing_this_round,now_cost,now_cost_rank,now_cost_rank_type,cost_change_event,cost_change_event_fall,cost_change_start,...,direct_freekicks_text,penalties_text,saves_per_90,clean_sheets_per_90,goals_conceded_per_90,starts_per_90,defensive_contribution_per_90,tackles,clearances_blocks_interceptions,recoveries
0,1,a,NaN,NaN,5.9,94,1.0,0,0.0,4,...,NaN,NaN,1.5,0.57,0.50,1.0,0.00,0,11,111
1,2,a,NaN,NaN,4.2,607,37.0,0,0.0,-3,...,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0,0,0
2,3,u,0.0,0.0,4.0,649,56.0,0,0.0,0,...,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0,0,0
3,4,a,NaN,NaN,3.9,736,87.0,0,0.0,-1,...,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0,0,0
4,5,i,0.0,0.0,6.3,72,2.0,0,0.0,3,...,NaN,NaN,0.0,0.64,0.45,1.0,9.09,12,88,15


In [81]:
df_small = df[[
    "id",
    "web_name",
    "minutes",
    "goals_scored",
    "assists",
    "expected_goals",
    "expected_assists",
    "expected_goal_involvements",
    "expected_goals_conceded",
    "defensive_contribution_per_90",
    "starts",
    "yellow_cards",
    "red_cards",
    "saves",
    "penalties_missed",
    "goals_conceded",
    "own_goals",
    "clean_sheets"
]].copy()

df_small.head()

,id,web_name,minutes,goals_scored,assists,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,defensive_contribution_per_90,starts,yellow_cards,red_cards,saves,penalties_missed,goals_conceded,own_goals,clean_sheets
0,1,Raya,1260,0,0,0.00,0.04,0.04,7.38,0.00,14,1,0,21,0,7,0,8
1,2,Arrizabalaga,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0
2,3,Hein,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0
3,4,Setford,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0
4,5,Gabriel,990,1,2,0.82,0.69,1.51,6.17,9.09,11,1,0,0,0,5,0,7


In [82]:
df_small["GI"] = df_small["goals_scored"] + df_small["assists"]

df_small = df_small.rename(columns={
    "expected_goal_involvements": "xGI",
})


df_small[["web_name", "GI", "xGI", "minutes"]]

,web_name,GI,xGI,minutes
0,Raya,0,0.04,1260
1,Arrizabalaga,0,0.00,0
2,Hein,0,0.00,0
3,Setford,0,0.00,0
4,Gabriel,3,1.51,990
...,...,...,...,...
753,Hee Chan,1,0.99,413
754,André,0,0.37,973
755,Cox,0,0.00,0
756,Fitzgerald,0,0.00,0


In [83]:
df_clean = df_small[df_small["minutes"] >= 300].copy()

df_clean.shape

(314, 19)

In [84]:
top_xg = df_clean.sort_values("xGI", ascending=False).head(9)

In [85]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Font
import matplotlib.font_manager as fm
font_path = "/content/Roboto-Regular.ttf"

fm.fontManager.addfont(font_path)
plt.rcParams["font.family"] = "Roboto"


In [ ]:
!pip install adjustText

In [ ]:
# Making scatter plot

from adjustText import adjust_text

# Size
plt.figure(figsize=(12,8), facecolor="#f2f2f2")

# Scatter plot
plt.scatter(
    df_clean["xGI"],
    df_clean["GI"],
    alpha=0.6,
    s=75,
    edgecolor="black",
    c="#e83410"
)

# Diagonal line
max_val = max(df_clean["xGI"].max(), df_clean["GI"].max())
plt.plot([0, max_val], [0, max_val],
         color="#4a4a4a",
         linestyle=":",
         linewidth=2,
         alpha=0.7,
         solid_capstyle="round"
         )

# Labels
plt.xlabel("xGI", fontsize=16,labelpad=14)
plt.ylabel("Goal Involvements", fontsize=16, labelpad=14)
plt.title("Goal Involvements vs. xGI", fontsize=32, pad=28)
plt.suptitle("PL 2025/26 (as of GW14)",
             fontsize=15,
             y=0.91,
             c="#4a4a4a"
             )

# Axes
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_color("#999999")
ax.spines["left"].set_color("#999999")
ax.spines["left"].set_linewidth(0.6)
ax.spines["bottom"].set_linewidth(0.6)
ax.set_facecolor("#f2f2f2")
ax.tick_params(
    axis='both',
    length=4,
    width=0.6,
    color="#999999",
    labelsize=14
)

# Axis increment
plt.xticks(range(0, int(max_val)+1, 2), fontsize=14, c="#4a4a4a")
plt.yticks(range(0, int(max_val)+1, 2), fontsize=14, c="#4a4a4a")

# Spacing
plt.subplots_adjust(
    left=0.1,
    right=0.9,
    bottom=0.1
)

# Dot labels
texts = []

for _, row in top_xg.iterrows():
    texts.append(
        plt.text(
            row["xGI"],
            row["GI"],
            row["web_name"],
            fontsize=12,
            color="#4a4a4a"
        )
    )

adjust_text(
    texts,
    ax=ax
)

plt.figtext(
    0.001, -0.01,
    "Data: olbauday on GitHub  •  Viz by @dmnarula_27",
    ha="left",
    fontsize=10,
    color="#666666"
)


plt.show()


In [ ]:
# Function to build reusable merge pipeline
def build_match_df(url_stats, url_players, url_teams, url_matches):
    # Load raw data
    df_stats   = pd.read_csv(url_stats)
    df_players = pd.read_csv(url_players)
    df_teams   = pd.read_csv(url_teams)
    df_matches = pd.read_csv(url_matches)

    # Keep certain columns
    player_cols = ["player_code", "player_id", "web_name", "team_code", "position"]
    team_cols   = ["code", "name", "short_name"]
    match_cols  = ["gameweek", "home_team", "away_team",
                   "match_id", "home_score", "away_score"]

    df_players = df_players[player_cols].copy()
    df_teams   = df_teams[team_cols].copy()
    df_matches = df_matches[match_cols].copy()

    # Fix type mismatch: home_team / away_team are floats, teams.code is int
    df_matches["home_team"] = df_matches["home_team"].astype(int)
    df_matches["away_team"] = df_matches["away_team"].astype(int)
    df_teams["code"]        = df_teams["code"].astype(int)

    # Merge stats + players (on player_id)
    df_sp = df_stats.merge(
        df_players,
        on="player_id",
        how="left"
    )

    # Merge in match info (on match_id)
    df_spm = df_sp.merge(
        df_matches,
        on="match_id",
        how="left"
    )

    # Merge in team info (on players.team_code → teams.code)
    df_final = df_spm.merge(
        df_teams,
        left_on="team_code",
        right_on="code",
        how="left"
    )

    # Clean up names
    df_final = df_final.rename(columns={
        "name": "team_name",
        "short_name": "team_short_name"
    }).drop(columns=["code"])

    return df_final

In [ ]:
# Load gameweek data into stats and players (currently GW15)
url_stats = "https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/refs/heads/main/data/2025-2026/By%20Gameweek/GW15/playermatchstats.csv"
url_players = "https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/refs/heads/main/data/2025-2026/By%20Gameweek/GW15/players.csv"
url_teams = "https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/refs/heads/main/data/2025-2026/By%20Gameweek/GW15/teams.csv"
url_matches = "https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/refs/heads/main/data/2025-2026/By%20Gameweek/GW15/matches.csv"

df_final = build_match_df(url_stats, url_players, url_teams, url_matches)
df_final.head()

In [ ]:
# Final cleaning
stats_cols = ["minutes_played", "goals", "assists",
              "total_shots", "xg", "xa", "shots_on_target"]
player_cols = ["player_code", "player_id", "web_name", "team_code", "position"]
match_cols  = ["gameweek", "home_team", "away_team",
                "match_id", "home_score", "away_score"]
team_cols   = ["team_name", "team_short_name"]

df_final_clean = df_final[stats_cols + player_cols + team_cols + match_cols]
df_final_clean.head()

In [ ]:
# Match Analysis Player xG vs. xA

# Select match ID
match_df = df_final_clean[
    df_final_clean["match_id"] == "25-26-prem-aston-villa-vs-arsenal"
]

# Building scatterplot

plt.figure(figsize=(12,8))

sns.scatterplot(
    data=match_df,
    x="xg",
    y="xa",
    s=90,
    alpha=0.8,
    hue="team_short_name",
    palette="Set1",
    edgecolor="black"
)

# Labels and legend
plt.xlabel("xG", fontsize=16)
plt.ylabel("xA", fontsize=16)
plt.title("xG vs xA Comparison", fontsize=32)
plt.suptitle("Aston Villa vs. Arsenal",
             fontsize=15,
             y=0.91,
             c="#4a4a4a")
plt.legend(title=None,)

# Axis formatting
ax = plt.gca()
ax.spines["bottom"].set_color("#999999")
ax.spines["left"].set_color("#999999")
ax.spines["left"].set_linewidth(0.6)
ax.spines["bottom"].set_linewidth(0.6)
ax.set_facecolor("#f2f2f2")
plt.tick_params(
    axis='both',
    which='both',
    length=4,
    width=0.6,
    color="#888888",
    labelsize=12
)


plt.grid(alpha=0.2)

# Remove top and right border
sns.despine()

plt.show()

